In [4]:
import os
import pathlib
import datetime
from pathlib import Path
from random import shuffle
import shutil


import pandas as pd

### Paths
Example path: **data/snr-15/COSTAS/signals_with_noise/costas1.png**

In [5]:
SOURCE_DIR = '/Users/Mjods/Documents/data/' # Full path
W_NOISE = 'signals_with_noise/'

### Filenames

In [6]:
METADATA = 'master_metadata.csv'

### Flags

In [69]:
# With test set
TEST_SET = True

# With noise 
NOISE = True

# Else binary
MULTICLASS = False 

### Parameters

In [70]:
test_split = 0.1
SNR = [-15,]
modulations = ['COSTAS', 'FMCW']

### Output structure

In [71]:
OUTPUT_DIR_NAME = 'training_dataset_{}/'.format(datetime.datetime.now())
dataset_names = ['train', 'test']

os.mkdir(OUTPUT_DIR_NAME)

for dataset in dataset_names:

    dataset_path = OUTPUT_DIR_NAME + dataset + '/'
    os.mkdir(dataset_path)

    if MULTICLASS:
        for modulation in modulations:
            os.mkdir(dataset_path + modulation)
    else:
        os.mkdir(dataset_path + 'SIGNAL')

    if NOISE:
        os.mkdir(dataset_path+'NOISE')

    

### Workspace

In [72]:
source = Path(SOURCE_DIR)
metadata = pd.read_csv(SOURCE_DIR + METADATA)

In [73]:
metadata.head()

,filename,modulation_type,snr,frequency_range,base_frequency
0,COSTAS_snr-15_0,COSTAS,-15,NaN,4.333388e+06
1,COSTAS_snr-15_1,COSTAS,-15,NaN,8.430625e+06
2,COSTAS_snr-15_2,COSTAS,-15,NaN,5.941570e+05
3,COSTAS_snr-15_3,COSTAS,-15,NaN,8.177337e+05
4,COSTAS_snr-15_4,COSTAS,-15,NaN,5.983043e+06


In [74]:
for snr in SNR:
    
    # Modulations
    for modulation in modulations:
    
        glob_path = '**/snr{}/{}/signals_with_noise/*.png'.format(snr, modulation)
        img_paths = [path for path in source.glob(glob_path)]
        
        # Shuffle
        shuffle(img_paths)
        
        # test split 
        length = len(img_paths)
        divider = int(length*(1-test_split))
        train = img_paths[:divider]
        test = img_paths [divider:]
        
        paths = [train,test]
        
        # Assert: number of datapoints! 
        if len(train) + len(test) != length:
            print("ERROR {} SIGNAL SPLIT: Total: {} Train: {} Test: {}".format(modulation, len(train), len(test)))
        
        # Copy the files
        for name, paths in zip(dataset_names, paths):
            for index,path in enumerate(paths):
                src = path
                dst = OUTPUT_DIR_NAME + name + '/'
                dst = dst + modulation if MULTICLASS is True else dst + 'SIGNAL/'
                shutil.copy(src,dst)
                

    # Noise
    glob_path = '**/snr{}/*.png'.format(snr)
    img_paths = [path for path in source.glob(glob_path)]
    shuffle(img_paths)
    
    length = len(img_paths)
    divider = int(length*(1-test_split))
    train = img_paths[:divider]
    test = img_paths [divider:]
    
    paths = [train,test]
    
     # Assert: number of datapoints! 
    if len(train) + len(test) != length:
        print("ERROR NOISE SPLIT: Total: {} Train: {} Test: {}".format(length,len(train), len(test)))
        
    
    for name, paths in zip(dataset_names, paths):
            for index,path in enumerate(paths):
                src = path
                dst = OUTPUT_DIR_NAME + name + '/' + 'NOISE'
                shutil.copy(src,dst)
        